In [1]:
import numpy as np
import decimal
import matplotlib.pyplot as plt
import sympy as sp

In [2]:
def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 

In [3]:
def parse_exp(expr_str , var_name='x') :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    
    def decimal_func(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
            
        result = expr.evalf(decimal.getcontext().prec,subs={x:x_decimal})
        return decimal.Decimal(str(result))
    
    return decimal_func 

In [4]:
def parse_derivative(expr_str , var_name='x' ) :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    deriv = sp.diff(expr)
    
    def decimal_derv(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
        result = deriv.evalf(decimal.getcontext().prec,subs = {x:x_decimal})
        return decimal.Decimal(str(result))
    return decimal_derv

In [ ]:
def ModifiedNewtonRaphson(x0, func_expr, tol=1e-7, max_iter=100,
                          significantFigs=14, rounding=True):
    """
    Modified Newton-Raphson method with second derivative correction.
    Uses the update formula:
    
        x_{n+1} = x_n - ( f(x_n) * f'(x_n) ) / ( (f'(x_n))^2 - f(x_n) * f''(x_n) )
    
    If the denominator becomes too small, the algorithm falls back
    to a standard Newton step: x_{n+1} = x_n - f(x_n) / f'(x_n).

    Returns:
        x_root        : final computed root
        relativeError : absolute error of last iteration
        num_iters     : number of iterations performed
        steps         : list of dictionaries with iteration history
    """

    # Set decimal precision and rounding mode
    intializeContext(significantFigs, rounding=rounding)

    # Convert initial guess to Decimal
    x_root = decimal.Decimal(x0)
    relativeError = decimal.Decimal("Infinity")

    # Parse function and derivatives
    func = parse_exp(func_expr)                   
    derivative = parse_derivative(func_expr)      

    # Build derivative expressions using sympy
    deriv_exp = str(sp.diff(sp.sympify(func_expr))) 
    d2_expr = str(sp.diff(sp.sympify(deriv_exp)))     
    second_derivative = parse_exp(d2_expr)          

    steps = []

    for i in range(max_iter):
        x_old = x_root

        # Evaluate f(x), f'(x), f''(x)
        fx = func(x_old)
        fpx = derivative(x_old)
        fppx = second_derivative(x_old)

        # Very small threshold to detect division issues
        eps = decimal.Decimal("1e-30")
        denominator = fpx**2 - fx * fppx

        # If denominator nearly zero → fallback to standard Newton step
        if abs(denominator) < eps:
            # Cannot proceed if derivative is zero
            if fpx == 0:
                break
            x_new = x_old - fx / fpx
        else:
            # Modified Newton–Raphson update
            x_new = x_old - (fx * fpx) / denominator

        # Compute absolute error
        if x_new != 0:
            relativeError = abs(x_new - x_old) / x_new
        else:
            relativeError = abs(x_old)

        # Save iteration information
        steps.append({
            "iter": i + 1,
            "x_old": x_old,
            "x_new": x_new,
            "error": relativeError
        })

        # Update current root estimate
        x_root = x_new

        # Stop if tolerance is satisfied
        if relativeError <= tol:
            break

    # Return results in the requested format
    return x_root, relativeError, len(steps), steps


In [17]:
x0 = 0.5
func_expr = "(x - 1)**3"

x_r, approx_error, num_iterations, steps = ModifiedNewtonRaphson(
    x0=x0,
    func_expr=func_expr,
    tol=1e-7,
    max_iter=20,
    significantFigs=14,
    rounding=True
)

print("Computed Root:", x_r)
print("approx_error: ", approx_error)
print("num_iterations: ", num_iterations)
for row in steps:
    print(row)

Computed Root: 1.0000000000000
approx_error:  0
num_iterations:  2
{'iter': 1, 'x_old': Decimal('0.5'), 'x_new': Decimal('1.0'), 'error': Decimal('0.5')}
{'iter': 2, 'x_old': Decimal('1.0'), 'x_new': Decimal('1.0000000000000'), 'error': Decimal('0')}


In [18]:
test_cases = [
    ("(x-1)**3", 0.5, 1),
    ("x**2 - 4", 3, 2),
    ("x**3 - 3*x**2 + 3*x - 1", 0, 1),
    ("x**3 + 2*x - 5", 1, 1.32827),
    ("x**4", 0.1, 0),
    ("sin(x)", 3, 3.14159265),
    ("exp(x) - 2", 0.5, 0.69314718),
]

for expr, x0, exact in test_cases:

    try:
        x_r, approx_error, num_iterations, steps = ModifiedNewtonRaphson(x0, expr, tol=1e-14, max_iter= 50, significantFigs=6)
        print(f"\nf(x) = {expr}, x0 = {x0}")
        print(f"  Computed root: {x_r}")
        print(f"  Expected root: {exact}")
        print(f"  Relative error: {abs(x_r - decimal.Decimal(str(exact)))}")
        print("approx_error: ", approx_error)
        print("num_iterations: ", num_iterations)
        for row in steps:
            print(row)
         
    except:
        print("Div by Zero")



f(x) = (x-1)**3, x0 = 0.5
  Computed root: 1.00000
  Expected root: 1
  Relative error: 0.00000
approx_error:  0
num_iterations:  2
{'iter': 1, 'x_old': Decimal('0.5'), 'x_new': Decimal('1.0'), 'error': Decimal('0.5')}
{'iter': 2, 'x_old': Decimal('1.0'), 'x_new': Decimal('1.00000'), 'error': Decimal('0')}

f(x) = x**2 - 4, x0 = 3
  Computed root: 2.00000
  Expected root: 2
  Relative error: 0.00000
approx_error:  0
num_iterations:  5
{'iter': 1, 'x_old': Decimal('3'), 'x_new': Decimal('1.84615'), 'error': Decimal('0.625003')}
{'iter': 2, 'x_old': Decimal('1.84615'), 'x_new': Decimal('1.99361'), 'error': Decimal('0.0739663')}
{'iter': 3, 'x_old': Decimal('1.99361'), 'x_new': Decimal('1.99999'), 'error': Decimal('0.00319002')}
{'iter': 4, 'x_old': Decimal('1.99999'), 'x_new': Decimal('2.00000'), 'error': Decimal('0.000005')}
{'iter': 5, 'x_old': Decimal('2.00000'), 'x_new': Decimal('2.00000'), 'error': Decimal('0')}

f(x) = x**3 - 3*x**2 + 3*x - 1, x0 = 0
  Computed root: 1
  Expected 